In [ ]:
import numpy as np
from scipy.integrate import nquad
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

np_val = 1.84085
ns_val = 1.75665
ni_val = 1.84475

def nint(y, ti, a, b, g):
    integrand = lambda x, ts: np.abs(
        x * np.exp(-((1 + b ** 2) * x ** 2 + y ** 2))
        * np.exp(-2 * x * y * np.cos(ts - ti))
        * np.sinc(g / 2 + a * ((1 - 2 * np_val / ns_val) * x ** 2 + (1 - 2 * np_val / ni_val) * y ** 2 + 2 * x * y * np.cos(ts - ti)))
    )
    result, _ = nquad(integrand, [[0, np.inf], [0, 2 * np.pi]])
    return result ** 2

def f(a, b, g):
    f_integrand = lambda y, ti: nint(y, ti, a, b, g)
    result, _ = nquad(f_integrand, [[0, np.inf], [0, 2 * np.pi]])
    return (4 / np.pi ** 4) * a * b * g * result

def compute_f_values(param_list):
    results = {}
    with ThreadPoolExecutor() as executor:
        future_to_params = {executor.submit(f, *params): params for params in param_list}
        with tqdm(total=len(future_to_params)) as pbar:
            for future in as_completed(future_to_params):
                params = future_to_params[future]
                try:
                    results[params] = future.result()
                except Exception as exc:
                    results[params] = f'Error: {exc}'
                pbar.update(1)
                print(f'f{params} = {results[params]}')
    return results

# Parameters to evaluate
param_list = [
    (2.7, 0.5, 3.2),
    (2.7, 1, 3.2),
    (2.7, np.sqrt(2), 3.2),
    (2.7, 1.75, 3.2),
    (2.7, 2, 3.2),
    (2.7, 2.5, 3.2),
    (2.7, 4, 3.2)
]

# Run computations in parallel
results = compute_f_values(param_list)

/Users/anani/Code/uq-qnn/.venv/lib/python3.9/site-packages/scipy/integrate/_quadpack_py.py:1272: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/var/folders/qr/scdb4s4n6dz5463spyd32yth0000gn/T/ipykernel_67338/3758089908.py:13: RuntimeWarning: overflow encountered in exp
  * np.exp(-2 * x * y * np.cos(ts - ti))
/var/folders/qr/scdb4s4n6dz5463spyd32yth0000gn/T/ipykernel_67338/3758089908.py:12: RuntimeWarning: invalid value encountered in scalar multiply
  x * np.exp(-((1 + b ** 2) * x ** 2 + y ** 2))
  0%|          